In [ ]:
%pip install geopandas as gpd
%pip install folium

In [2]:
import geopandas as gpd
import folium

In [3]:
# source: https://data.grandlyon.com/portail/fr/jeux-de-donnees/contours-iris-grande-echelle-metropole-lyon/telechargements
iris = gpd.read_file("data/IRIS.json")

# source: https://transport.data.gouv.fr/resources/65812?locale=en
stops = gpd.read_file('data/stops.geojson')

In [4]:
iris.head()

,id,codeiris,libelle,type,typelibelle,commune,insee,gid,geometry
0,ter_territoire.teriris_ge_latest.1,690030000,Albigny-sur-Saône,Z,Commune non irisée,Albigny-sur-Saône,69003,1,"POLYGON ((4.81744 45.85602, 4.81762 45.85619, ..."
1,ter_territoire.teriris_ge_latest.2,690290101,Les Genêts-Hôpitaux,H,Habitat,Bron,69029,2,"POLYGON ((4.89185 45.73853, 4.89188 45.73910, ..."
2,ter_territoire.teriris_ge_latest.3,690290102,Gendarmerie-Garenne,H,Habitat,Bron,69029,3,"POLYGON ((4.90602 45.74374, 4.90615 45.74416, ..."
3,ter_territoire.teriris_ge_latest.4,690290103,Dubœuf-Camille Rousset,H,Habitat,Bron,69029,4,"POLYGON ((4.90224 45.73569, 4.90225 45.73570, ..."
4,ter_territoire.teriris_ge_latest.5,690290104,Ferdinand Buisson Église,H,Habitat,Bron,69029,5,"POLYGON ((4.90969 45.74282, 4.91026 45.74281, ..."


In [5]:
# calculate the number of stops in each IRIS area
iris['stops'] = [stops.within(row.geometry).sum() for _, row in iris.iterrows()]

# calculate the number of stops with wheelchair access in each IRIS area
iris['accessibility'] = [stops.within(row.geometry) & (stops.wheelchair_boarding == 'available') for _, row in iris.iterrows()]

In [6]:
# see https://python-visualization.github.io/folium/latest/advanced_guide/colormaps.html for info on Folium colormaps
colormap = folium.LinearColormap(['red', 'yellow', 'green'], vmin=0, vmax=1)

colors = []
opacities = []
for _, row in iris.iterrows():
    # if a region has no stops, make it transparent (gray is just to avoid indexing errors below)
    if row.stops == 0:
        colors.append('gray')
        opacities.append(0)
    # otherwise, color it according to the fraction of stops with wheelchair access
    else:
        colors.append(colormap(row.accessibility.sum() / row.stops))
        opacities.append(0.7)


In [7]:
# add a style column to the iris dataframe
iris['style'] = [{
                    "fillColor": colors[row.gid-1],     # get the color for the region with geometry id = gid (index starts at 1)
                    "color": "black",
                    "weight": "1",
                    "fillOpacity": opacities[row.gid-1] # get the opacity for the region with geometry id = gid (index starts at 1)
                } for _, row in iris.iterrows()]

# add a tooltip column to the iris dataframe
iris['tooltip'] = [f"Stops: {row.stops}<br>Accessible Stops: {row.accessibility.sum()}" for _, row in iris.iterrows()]


In [8]:
# remove stops column and accessibility column to make folium happy
iris = iris.drop(columns=['stops', 'accessibility'])
iris.head()

,id,codeiris,libelle,type,typelibelle,commune,insee,gid,geometry,style,tooltip
0,ter_territoire.teriris_ge_latest.1,690030000,Albigny-sur-Saône,Z,Commune non irisée,Albigny-sur-Saône,69003,1,"POLYGON ((4.81744 45.85602, 4.81762 45.85619, ...","{'fillColor': '#7fc000ff', 'color': 'black', '...",Stops: 20<br>Accessible Stops: 15
1,ter_territoire.teriris_ge_latest.2,690290101,Les Genêts-Hôpitaux,H,Habitat,Bron,69029,2,"POLYGON ((4.89185 45.73853, 4.89188 45.73910, ...","{'fillColor': '#a0d000ff', 'color': 'black', '...",Stops: 35<br>Accessible Stops: 24
2,ter_territoire.teriris_ge_latest.3,690290102,Gendarmerie-Garenne,H,Habitat,Bron,69029,3,"POLYGON ((4.90602 45.74374, 4.90615 45.74416, ...","{'fillColor': '#bfe000ff', 'color': 'black', '...",Stops: 16<br>Accessible Stops: 10
3,ter_territoire.teriris_ge_latest.4,690290103,Dubœuf-Camille Rousset,H,Habitat,Bron,69029,4,"POLYGON ((4.90224 45.73569, 4.90225 45.73570, ...","{'fillColor': '#55ab00ff', 'color': 'black', '...",Stops: 6<br>Accessible Stops: 5
4,ter_territoire.teriris_ge_latest.5,690290104,Ferdinand Buisson Église,H,Habitat,Bron,69029,5,"POLYGON ((4.90969 45.74282, 4.91026 45.74281, ...","{'fillColor': '#7fc000ff', 'color': 'black', '...",Stops: 8<br>Accessible Stops: 6


In [9]:
# Create a map centered on the mean coordinates of the stops
m = folium.Map(location=[45.75, 4.85], zoom_start=13)

# add color legend
colormap.caption = 'Fraction of accessible stops in the region'
colormap.add_to(m)

# Add the iris regions to the map with the style column
folium.GeoJson(
    iris,
    name='geojson',
    style_function=lambda x: x['properties']['style'],
    tooltip=folium.GeoJsonTooltip(fields=['libelle', 'tooltip'], aliases=['Region', 'Information'])
).add_to(m)

# save the map
m.save('out/iris_map.html')

# uncomment to display the map
# m